In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from joblib import dump

In [3]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head(5)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
# from pandas_profiling import ProfileReport
# prof = ProfileReport(df)
# prof.to_file('HeartFailure_report.html')

In [5]:
df1 = df.drop(columns=['creatinine_phosphokinase','diabetes'])

In [6]:
x_train,x_test,y_train,y_test = train_test_split(df1.drop(columns=['DEATH_EVENT']),df1['DEATH_EVENT'],test_size=0.2,random_state=42)

In [7]:
x_test.head()

,age,anaemia,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
281,70.0,0,40,0,51000.0,2.7,136,1,1,250
265,50.0,1,35,0,362000.0,0.9,140,1,1,240
164,45.0,0,30,0,334000.0,1.1,139,1,0,129
9,80.0,1,35,1,388000.0,9.4,133,1,1,10
77,42.0,0,40,0,237000.0,1.2,140,1,0,74


In [8]:
# import seaborn as sns

In [9]:
# sns.heatmap(pd.crosstab(df['sex'],df['smoking']))

In [10]:
scaler = MinMaxScaler()

In [11]:
x_train_platelets = scaler.fit_transform(x_train[['platelets']])
x_test_platelets = scaler.fit_transform(x_test[['platelets']])
x_train_time = scaler.fit_transform(x_train[['time']])
x_test_time = scaler.fit_transform(x_test[['time']])

In [12]:
model = RandomForestClassifier(max_depth=10, random_state=42)

In [13]:
x_train_new = x_train.drop(columns=['platelets','time'])
x_test_new = x_test.drop(columns=['platelets','time'])

In [14]:
x_train_transformed = np.concatenate((x_train_new,x_train_platelets,x_train_time),axis=1)
x_test_transformed = np.concatenate((x_test_new,x_test_platelets,x_test_time),axis=1)

In [15]:
# x_train_transformed.shape

In [16]:
model.fit(x_train_transformed,y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [17]:
x_train_transformed.shape

(239, 10)

In [18]:
y_pred = model.predict(x_test_transformed)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6833333333333333

In [20]:
x_platelets = scaler.fit_transform(df1[['platelets','time']])

In [21]:
x_new = df1.drop(columns=['platelets','time','DEATH_EVENT'])

In [22]:
x_transformed = np.concatenate((x_new,x_platelets),axis=1)

In [23]:
x_transformed[1]

array([5.50000000e+01, 0.00000000e+00, 3.80000000e+01, 0.00000000e+00,
       1.10000000e+00, 1.36000000e+02, 1.00000000e+00, 0.00000000e+00,
       2.88832622e-01, 7.11743772e-03])

In [24]:
np.mean(cross_val_score(model,x_transformed,df1['DEATH_EVENT'],scoring='accuracy',cv=10))

0.7689655172413794

In [25]:
dump(model,'HF Prediction Model.joblib')

['HF Prediction Model.joblib']

In [28]:
df3 = pd.DataFrame(x_transformed, columns=[['age', 'anaemia', 'ejection_fraction', 'high_blood_pressure',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'platelets', 'time']])

In [29]:
df3.describe()

,age,anaemia,ejection_fraction,high_blood_pressure,serum_creatinine,serum_sodium,sex,smoking,platelets,time
count,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.000000
mean,60.833893,0.431438,38.083612,0.351171,1.39388,136.625418,0.648829,0.32107,0.288833,0.449327
std,11.894809,0.496107,11.834841,0.478136,1.03451,4.412477,0.478136,0.46767,0.118565,0.276207
min,40.000000,0.000000,14.000000,0.000000,0.50000,113.000000,0.000000,0.00000,0.000000,0.000000
25%,51.000000,0.000000,30.000000,0.000000,0.90000,134.000000,0.000000,0.00000,0.227179,0.245552
50%,60.000000,0.000000,38.000000,0.000000,1.10000,137.000000,1.000000,0.00000,0.287186,0.395018
75%,70.000000,1.000000,45.000000,1.000000,1.40000,140.000000,1.000000,1.00000,0.337495,0.708185
max,95.000000,1.000000,80.000000,1.000000,9.40000,148.000000,1.000000,1.00000,1.000000,1.000000


In [31]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000
